In [1]:
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/Users/phillipguo/opt/anaconda3/envs/pyt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [63]:
%%capture
from transformers import AutoTokenizer, AutoModel, pipeline, OPTForCausalLM

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained("facebook/opt-125m", output_attentions=True, output_hidden_states=True)

generator = pipeline('text-generation', model="facebook/opt-125m")
#generator = pipeline('text-generation', model=model)


In [36]:
generator("Bush did 9/11 because")

[{'generated_text': "Bush did 9/11 because he was a terrorist.\nI'm not sure if you're being"}]

In [52]:
# for name, param in model.named_parameters():
for name, param in model.named_parameters():
    print(name)
    # print(param.shape)

# print(model.decoder.layers[0].self_attn)

model.decoder.embed_tokens.weight
model.decoder.embed_positions.weight
model.decoder.final_layer_norm.weight
model.decoder.final_layer_norm.bias
model.decoder.layers.0.self_attn.k_proj.weight
model.decoder.layers.0.self_attn.k_proj.bias
model.decoder.layers.0.self_attn.v_proj.weight
model.decoder.layers.0.self_attn.v_proj.bias
model.decoder.layers.0.self_attn.q_proj.weight
model.decoder.layers.0.self_attn.q_proj.bias
model.decoder.layers.0.self_attn.out_proj.weight
model.decoder.layers.0.self_attn.out_proj.bias
model.decoder.layers.0.self_attn_layer_norm.weight
model.decoder.layers.0.self_attn_layer_norm.bias
model.decoder.layers.0.fc1.weight
model.decoder.layers.0.fc1.bias
model.decoder.layers.0.fc2.weight
model.decoder.layers.0.fc2.bias
model.decoder.layers.0.final_layer_norm.weight
model.decoder.layers.0.final_layer_norm.bias
model.decoder.layers.1.self_attn.k_proj.weight
model.decoder.layers.1.self_attn.k_proj.bias
model.decoder.layers.1.self_attn.v_proj.weight
model.decoder.layers

In [ ]:
# string representing module attribute of model
# name is name of parameter with parameter tensor
# def get_module_attr(model, name):
    # if name contains a number
    

In [66]:
import torch.nn.utils.prune 

# param_name is like "weight", tells which part of module to access for tensor
def init_mask(module, param_name):
    # mask=torch.ones_like(module.weight)
    # mask=torch.ones_like(getattr(module, param_name))
    mask=torch.zeros_like(getattr(module, param_name))

    mask_applier = torch.nn.utils.prune.CustomFromMask(mask)
    # apply the mask to the module
    mask_applier.apply(module=module, name=param_name, mask=mask)
    
# layer = model.get_decoder().layers[0].fc1
layer = model.get_decoder().layers[0].self_attn.out_proj

# the two actual tensors but in module form
weight_layer = layer.weight
bias_layer = layer.bias

init_mask(layer, "weight")

# torch.nn.utils.prune.CustomFromMask(mask)

In [55]:
# A class to mask a transformer's layers
class Masker:
    # initialize the masks
    def __init__(model):
        # rename all of the modules so you can access them later
        for n, m in model.named_modules():
            m.auto_name = n
        

        # iterate through all 

In [70]:
input1 = tokenizer("Hello, my dog is cute", return_tensors="pt", padding="max_length", truncation=True)
input2 = tokenizer("What the fuck did you just fucking say about me, you little bitch?", return_tensors="pt", padding="max_length", truncation=True)
output = model.generate(input1.input_ids, max_length=300, num_return_sequences=1, temperature=0.5, top_p=0.95)
tokenizer.decode(output[0], skip_special_tokens=True)

"Hello, my dog is cute.\nI'm sorry, I'm sorry, I'm sorry, I'm sorry, I'm sorry, I'm sorry, I'm sorry, I'm sorry, I'm sorry, I'm sorry, I'm sorry, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I'm, I"